# STEP1


1.   刪除取一年中每月淨利為負的(若標準過高則再作修正)-刪除極端值
2.   所有1~3月本益比平均or當季本益比 的 describe()-刪除極端值




### 匯入套件

In [ ]:
pip install jmespath

In [ ]:
import requests
import json
#可從json格式中取得資料(json格式定位資料位置)
import jmespath
import pandas as pd
import time
import re

### 目標資訊

In [ ]:
#目標股票代碼
stocklist=["1201","1203","1210","1213","1215","1216","1217","1218","1219","1220","1225","1227","1229","1231","1232","1233","1234","1235","1236","1256","1702","1737"]
#目標時間
datelist=[]
for year in range(2019,2022):
    for month in range(1,13):
        date=str(year)+str("%02d" % month)+"01"
        if date == '20210401':
          break
        else:
          datelist.append(date)

### 爬蟲(本益比資訊)

In [ ]:
def get_peratio(stock,date):
  #創建存放資訊dict
  data={"STOCK":[],"年分":[],"本益比":[]} 

  for date in datelist:
    try:
      for stock in stocklist:
        #替代成所需的股票代碼及時間
        request_url = f'https://www.twse.com.tw/exchangeReport/BWIBBU?response=json&date={date}&stockNo={stock}' 
        res = requests.get(request_url)
        resinfo = json.loads(res.text)
        #定位所需資訊
        peratio=jmespath.search('data[:][3][]',resinfo) 
        for peratioidx in range(len(peratio)):
          #若該欄位並無放置資訊或資訊非浮點數格式會出錯，因此遇到錯誤則跳過
          try:
            #將資訊填入相對應位置
            data['本益比'].append(float(peratio[peratioidx]))
            data['STOCK'].append(stock)
            data['年分'].append((re.findall(r'^2\w{3}',date)[0]))
          except TypeError:
            print(peratioidx)
            pass
        #避免過快取得網頁資訊會被封鎖ip
        time.sleep(3)
    except ValueError:
      stocklist.remove(stock)
  return data

In [21]:
df = pd.DataFrame(get_peratio(stocklist,datelist))
#將欄位整理成以股票及年為群組，分別使用max﹑min及mean的方式
df1 = pd.concat([df.groupby(['STOCK','年分']).mean(), df.groupby(['STOCK','年分']).min(), df.groupby(['STOCK','年分']).max()], axis=1)
df1.columns = ['平均本益比','最小本益比','最大本益比']

In [22]:
df1

平均本益比  最小本益比   最大本益比
STOCK 年分                             
1201  2019    8.505777   6.08   14.48
      2020   26.831673   6.37   45.59
      2021   27.750727  20.33   29.73
1203  2019   13.866033  12.96   14.84
      2020   16.171959  13.65   19.74
      2021   18.756000  17.90   21.10
1210  2019   14.294215  10.94   18.18
      2020   13.945918  11.03   15.71
      2021   14.266909  13.46   15.45
1215  2019   15.831446  13.30   21.33
      2020   12.234286  10.18   14.15
      2021   11.214364  10.68   12.39
1216  2019   20.734587   9.64   25.62
      2020   20.654041  17.76   23.31
      2021   18.632545  17.57   19.26
1217  2019   69.636157  17.03  150.40
      2020   46.414898  19.00   72.40
      2021   21.536182  19.17   24.76
1218  2019   20.025413  18.91   21.93
      2020   17.775102  14.56   24.06
      2021   17.757818  16.49   18.89
1220  2019   24.086529  18.04   28.85
      2020   14.755061  12.47   19.02
      2021   12.836727  11.98   14.27
1225  2019   55.476157  22.05   94.94
      2020   22.616816  19.60   26.42
      2021   25.323273  21.96   31.40
1227  2019   17.320513  15.45   20.45
      2020   17.211633  14.83   20.37
      2021   14.866000  14.19   16.55
1229  2019   13.458264  11.65   15.02
      2020   18.614816  11.27   26.85
      2021   20.119273  19.10   22.04
1231  2019   13.853719  12.30   15.32
      2020   13.745102  10.73   15.46
      2021   13.263273  12.58   15.80
1232  2019   19.129711  16.77   20.83
      2020   19.209265  14.50   21.53
      2021   19.095818  17.08   20.38
1233  2019   20.251529  18.51   23.02
      2020   31.413143  17.84   47.70
      2021   47.535818  44.80   55.87
1234  2019   17.583884  15.82   19.19
      2020   15.916612  13.83   17.41
      2021   15.013273  14.66   15.66
1235  2019   96.548678   9.50  487.50
      2020  112.548531  26.13  406.67
      2021   34.155455  31.15   44.85
1256  2019   12.648182  10.35   15.80
      2020   14.875918  10.37   19.93
      2021   16.422000  13.75   19.97
1702  2019   13.517149  12.54   15.11
      2020   12.134694   9.02   13.93
      2021   11.666909  11.07   12.42
1737  2019   16.304793  13.14   20.21
      2020   19.581306  15.77   21.59
      2021   20.285455  18.31   20.79

In [67]:
df1.to_csv('stockstep1.csv')

In [68]:
df1 = pd.read_csv('stockstep1.csv')

# STEP 2

*   算出合理區間
*   Link：https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1201&SYEAR=2020&SSEASON=4&REPORT_ID=C





### 找出EPS

In [49]:
#下載所需csv
!gdown --id '1qyBFxEfEF9jDHrTvG4i5SlNWtdCr_soC'
!gdown --id '17iFuLmAL3jOocE0vMZKqMkERWQnTR85B'
!gdown --id '19bqdpigtGm1J2BsV3wJgoJP98ulWMzxv'

# 合併3個檔案
read = pd.read_csv('stock_4.csv')
read.to_csv('stockstep2_ori.csv')
for i in range(8, 10):
    read = pd.read_csv('stock_{}.csv'.format(i))
    read.to_csv('stockstep2_ori.csv',header=False, mode='a+')

Downloading...
From: https://drive.google.com/uc?id=1qyBFxEfEF9jDHrTvG4i5SlNWtdCr_soC
To: /content/stock_4.csv
100% 169k/169k [00:00<00:00, 62.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=17iFuLmAL3jOocE0vMZKqMkERWQnTR85B
To: /content/stock_8.csv
100% 169k/169k [00:00<00:00, 63.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=19bqdpigtGm1J2BsV3wJgoJP98ulWMzxv
To: /content/stock_9.csv
100% 153k/153k [00:00<00:00, 57.0MB/s]


In [52]:
df2 = pd.read_csv('stockstep2_ori.csv', index_col='Unnamed: 0')

In [54]:
#利用上方剩下的股票代碼刪除新csv中不需要的股票代碼
stocklist = [str(i) for i in list(set(df1.index.get_level_values('STOCK')))]
df2 = df2[df2['公司代號'].isin(stocklist)]

In [55]:
df2 = df2[[x not in list(set(df2[df2['基本每股盈餘（元）']<0]['公司代號'].tolist())) for x in df2['公司代號']]]

In [58]:
#資料為兩年即一記得每股盈餘，因此加總/2.25
df2 = df2.groupby(['公司代號']).sum()/2.25
#刪除不需要的欄位
df2 = df2.drop(df2.columns[:-1], axis=1)
#重新命名欄位名稱
df2 = df2.rename(columns={'公司代號': 'STOCK','基本每股盈餘（元）': 'EPS'})

In [61]:
df2.index.names = ['STOCK']

In [71]:
df2

,EPS
STOCK,
1201,0.770370
1203,0.845432
1210,1.566420
1215,2.585679
1216,1.598025
1217,0.140247
1218,0.649877
1220,0.414815
1225,0.825679


### 算出合理區間

In [72]:
#以STOCK欄位合併兩張資料表
df = pd.merge(df1,df2,on='STOCK')
df = pd.concat([df.groupby(['STOCK']).min().filter(['STOCK','最小本益比'], axis=1), df.groupby(['STOCK']).max().filter(['STOCK','最大本益比'], axis=1), df.groupby(['STOCK']).mean().filter(['STOCK','平均本益比','EPS'], axis=1)], axis=1)

In [73]:
df

,最小本益比,最大本益比,平均本益比,EPS
STOCK,,,,
1201,6.08,45.59,21.029393,0.770370
1203,12.96,21.10,16.264664,0.845432
1210,10.94,18.18,14.169014,1.566420
1215,10.18,21.33,13.093365,2.585679
1216,9.64,25.62,20.007058,1.598025
1217,17.03,150.40,45.862412,0.140247
1218,14.56,24.06,18.519444,0.649877
1220,11.98,28.85,17.226106,0.414815
1225,19.60,94.94,34.472082,0.825679


In [74]:
#算出股價區間
df['便宜股價'] = df['最小本益比']*df['EPS']
df['安全股價'] = ((df['最小本益比']+df['平均本益比'])/2)*df['EPS']
df['合理股價'] = df['平均本益比']*df['EPS']
df['觀察股價'] = ((df['最大本益比']+df['平均本益比'])/2)*df['EPS']
df['昂貴股價'] = df['最大本益比']*df['EPS']

In [75]:
#取小數點後第5位
df = df.round(5)

In [76]:
df.to_csv('stockstep2.csv')